![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20RoBertaForTokenClassification.ipynb)

## Import RoBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.3.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import RoBERTa models trained/fine-tuned for token classification via `RobertaForTokenClassification` or `TFRobertaForTokenClassification`. These models are usually under `Token Classification` category and have `roberta` in their labels
- Reference: [TFRobertaForTokenClassification](https://huggingface.co/transformers/model_doc/roberta.html#tfrobertafortokenclassification)
- Some [example models](https://huggingface.co/models?filter=roberta&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.


In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [philschmid/distilroberta-base-ner-wikiann-conll2003-3-class](https://huggingface.co/philschmid/distilroberta-base-ner-wikiann-conll2003-3-class) model from HuggingFace as an example
- In addition to `TFRobertaForTokenClassification` we also need to save the `RobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFRobertaForTokenClassification, RobertaTokenizer 
import tensorflow as tf

MODEL_NAME = 'philschmid/distilroberta-base-ner-wikiann-conll2003-3-class'

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFRobertaForTokenClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFRobertaForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 637280
-rw-r--r--  1 maziyar  staff       1034 Dec 15 17:51 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 15 17:51 saved_model
-rw-r--r--  1 maziyar  staff  326280552 Dec 15 17:51 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 9672
drwxr-xr-x  2 maziyar  staff       64 Dec 15 17:51 assets
-rw-r--r--  1 maziyar  staff       52 Dec 15 17:51 fingerprint.pb
-rw-r--r--  1 maziyar  staff    89042 Dec 15 17:51 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  4854186 Dec 15 17:51 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 15 17:51 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 2864
-rw-r--r--  1 maziyar  staff  456318 Dec 15 17:50 merges.txt
-rw-r--r--  1 maziyar  staff     280 Dec 15 17:50 special_tokens_map.json
-rw-r--r--  1 maziyar  staff    1388 Dec 15 17:50 tokenizer_config.json
-rw-r--r--  1 maziyar  staff  999355 Dec 15 17:50 vocab.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.json` and `merges.txt` files from the tokenizer
- all we need is to first convert `vocab.json` to `vocab.txt` and copy both `vocab.txt` and `merges.txt` into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in tokenizer.get_vocab().keys():
        f.write("%s\n" % item)

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}
!cp {MODEL_NAME}_tokenizer/merges.txt {asset_path}

In [ ]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 1704
-rw-r--r--  1 maziyar  staff      37 Dec 15 17:51 labels.txt
-rw-r--r--  1 maziyar  staff  456318 Dec 15 17:51 merges.txt
-rw-r--r--  1 maziyar  staff  407065 Dec 15 17:51 vocab.txt


## Import and Save RobertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.5
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.5


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `RoBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = RoBertaForTokenClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your RoBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 646384
drwxr-xr-x  6 maziyar  staff        192 Dec 15 17:51 fields
drwxr-xr-x  6 maziyar  staff        192 Dec 15 17:51 metadata
-rw-r--r--  1 maziyar  staff  330946755 Dec 15 17:51 roberta_classification_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForTokenClassification model 😊 

In [ ]:
tokenClassifier_loaded = RoBertaForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-LOC', 'I-PER', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["My name is Clara and I live in Berkeley, California."], ['My name is Wolfgang and I live in Berlin.']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|My name is Clara ...|[O, O, O, B-PER, ...|
|My name is Wolfga...|[O, O, O, B-PER, ...|
+--------------------+--------------------+



That's it! You can now go wild and use hundreds of `RoBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀 
